In [ ]:
try:
    import tweepy  
except ImportError:
    %pip install tweepy
    import tweepy

In [ ]:
try:
    import jsonpickle
except ImportError:
    %pip install jsonpickle
    import jsonpickle

try:
    from newsapi import NewsApiClient
except ImportError:
    %pip install NewsApiClient
    from newsapi import NewsApiClient

In [ ]:
import pandas as pd
import json
import datetime
import os

In [ ]:
class tweets():
    # This function will identify the latest tweet extract file created in the folder and then identify the id
    # of the last tweet.
    def find_lastTweet(self):
        path = os.getcwd()
        folder = "Data Dump/**/FP1_Tweets*.txt"
        fullpath = os.path.join(path, folder)
        list_of_files = glob.glob(fullpath, recursive = True)  
        if not list_of_files:                
            self.lastTweet = None                      
        else:
            latest_file = max(list_of_files, key=os.path.getctime)
            f = open(latest_file,"r")
            #Read the first tweet
            self.lastTweet = int (eachline.split(",")[1].split(":")[1].strip()) - 1 if eachline else None
            f.close()
    
    def fetch_tweets(self,
                     keyword,
                     ckey, 
                     csecret, 
                     atoken, 
                     asecret, 
                     currentTime,
                     currDate,
                     n_tweets):

        print('\nINFO!! Extracting tweets for ' + keyword)
        print('----- Establishing connection')
        auth = tweepy.OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        api = tweepy.API(auth,wait_on_rate_limit=True)
        extractedCnt = 0
        filePart = 1
       
        if not (os.path.isdir("Data Dump/"+ currDate)):
            os.mkdir("Data Dump/"+ currDate)
        # Open the file
        f = open("Data Dump/" + currDate + "/FP1_tweets_"+currentTime+"_"+str(filePart)+".txt", "a")
        for tweet in tweepy.Cursor(api.search,q="#Trump",count=n_tweets,
                           lang="en",tweet_mode='extended',result_type='recent',since_id = '1340628236972134400').items():

            extractedCnt+=1
            #if (extractedCnt > n_tweets):
            #    break
            if (extractedCnt % 100 == 0):
                print("      Tweets extracted so far is {0}".format(extractedCnt))
            f.write(jsonpickle.encode(tweet, unpicklable=False) +'\n')
            if (extractedCnt % 200 == 0):
                f.close()
                filePart += 1
                f = open("Data Dump/" + currDate + "/FP1_tweets_"+currentTime+"_"+str(filePart)+".txt", "a")
        f.close()
        print('INFO!! Finished extracting tweets for ' + keyword + '\n')
        
        return

In [ ]:
class news_bytes():
    def fetch_news(self,
                   keyword, 
                   api_key,
                   currentTime,
                   currDate):
        print('\nINFO!! Extracting news for ' + keyword)
        api = NewsApiClient(api_key=api_key)
        temp = api.get_everything(q = keyword)

        if not (os.path.isdir("Data Dump/"+ currDate)):
            os.mkdir("Data Dump/"+ currDate)
        # Open the file
        f = open("Data Dump/" + currDate + "/FP1_news_"+currentTime+".txt", "w")
        f.write(jsonpickle.encode(temp.get('articles'), unpicklable=False) +'\n')
        f.close()
        print('INFO!! Finished extracting news for ' + keyword + '\n')
        
        return 

In [ ]:
if __name__ == '__main__':
    keywords = ['trump','USElections']

    #Fetch Current time for appending to file name
    now = datetime.datetime.now()
    currentTime=str(now.year)+"_"+str(now.month).zfill(2)+"_"+str(now.day).zfill(2)+"_"+str(now.hour).zfill(2)+"_"+str(now.minute).zfill(2)
    currDate = str(now.year) + str(now.month).zfill(2) + str(now.day).zfill(2)
    print(currentTime)
    
    # News API Key
    api_key ='2d316a0ecc7e47768d88bc7201652b33'
    newsBytesObj = news_bytes()
    
    tweetObj = tweets()
    #Number of tweets to be fetched
    n_tweets = 100    
    # Twitter secrets
    ckey = 'lNay2TfnIw61ptsbUdqcmeX0y'
    csecret = '1NiVM6dKY7GrYnI4G06s2BoA1Qli7BZFC1ThlQ5JXB29drhEQA'
    atoken = '1284353261563068417-sJ3mBP10v0ZKa6v7KY8uDYRAl92xE7'
    asecret = 'nxQgeIyMbr5WffFeYFYS0odIKnfwPVUglpeBrRAK9PkGj'
    
    
    
    for keyword in keywords:
        print('INFO!! Extracting data for ' + keyword)
        newsBytesObj.fetch_news(keyword, api_key, currentTime, currDate)    
        tweetObj.fetch_tweets(keyword, ckey, csecret, atoken, asecret, currentTime, currDate, n_tweets)

      Tweets extracted so far is 9000
      Tweets extracted so far is 9100
      Tweets extracted so far is 9200
